In [ ]:
import sqlite3
# создание таблиц
connection = sqlite3.connect('task1.sqlite')
cursor = connection.cursor()

cursor.execute('''CREATE TABLE actors (
                id INTEGER PRIMARY KEY,
                name TEXT )
''')

cursor.execute('''CREATE TABLE movies (
                id INTEGER PRIMARY KEY,
                title TEXT )
''')

cursor.execute('''CREATE TABLE actor_movie (
               actor_id INTEGER,
               movie_id INTEGER,
               FOREIGN KEY (actor_id) REFERENCES actors(id),
               FOREIGN KEY (movie_id) REFERENCES movies(id) )
''')

connection.commit()
connection.close()

In [13]:
connection_netflix = sqlite3.connect('netflix.sqlite')
cursor_netflix = connection_netflix.cursor()
cursor_netflix.execute('SELECT title, "cast" FROM netflix_titles')
data = cursor_netflix.fetchall()
connection_netflix.close()


In [ ]:
import re
update_data = []
unique_actors = set()
for i in data:
    film, actors = i
    cast = re.split(r' ,|, |,', actors)
    unique_actors = unique_actors.union(set(cast))
    update_data.append((film, cast))

In [ ]:
connection_actors = sqlite3.connect('task1.sqlite')
cursor_actors = connection_actors.cursor()
for actor in unique_actors:
    cursor_actors.execute("INSERT INTO actors VALUES (NULL, ?)", (actor,))
connection_actors.commit()
connection_actors.close()

In [ ]:
connection_films = sqlite3.connect('task1.sqlite')
cursor_movies = connection_films.cursor()
for i in update_data:
    film, actors = i
    cursor_movies.execute("INSERT INTO movies VALUES (NULL, ?)", (film,))
connection_films.commit()
connection_films.close()

In [ ]:
connection_actor_film = sqlite3.connect('task1.sqlite')
cursor_actors = connection_actor_film.cursor()
curso_films = connection_actor_film.cursor()
cursor_actor_film = connection_actor_film.cursor()
for i in update_data:
    film, actors = i
    cursor_actor_film.execute('SELECT id FROM movies WHERE title = ?', (film,))
    unique_film = cursor_actor_film.fetchone()
    for actor in actors:
        cursor_actors.execute('SELECT id FROM actors WHERE name = ?', (actor,))
        unique_actor = cursor_actors.fetchone()
        cursor_actor_film.execute("INSERT INTO actor_movie (actor_id, movie_id) VALUES (?, ?)", (unique_actor[0], unique_film[0]))
connection_actor_film.commit()
connection_actor_film.close()

In [ ]:
connection_find_actors = sqlite3.connect('task1.sqlite')
cursor_find_actors = connection_find_actors.cursor()
cursor_find_actors.execute('''
    SELECT a1.name AS actor1_name, a2.name AS actor2_name, COUNT(*) AS movies_together
    FROM actor_movie am1
    JOIN actor_movie am2 ON am1.movie_id = am2.movie_id AND am1.actor_id < am2.actor_id
    JOIN actors a1 ON am1.actor_id = a1.id
    JOIN actors a2 ON am2.actor_id = a2.id
    GROUP BY actor1_name, actor2_name
    ORDER BY movies_together DESC
    LIMIT 1
''')
actors = cursor_find_actors.fetchone()
print(actors[0])
print(actors[1])
connection_find_actors.close()